In [1]:
from keras.models import  load_model
from keras.utils import np_utils
import glob
import numpy as np
import os
import sed_eval
import dcase_util
import pickle
#%run shared_functions.ipynb

from PIL import Image

Using TensorFlow backend.


In [2]:

def getStackFeatMat(scene):
    with open(scene, "rb" ) as scenesample:
        l,fv = pickle.load(scenesample)
    return fv,l

def reduceLabels(labels):
    r,c=labels.shape 
    npOfLabels=np.zeros(c)
    for j in range(0,c):
        if np.sum(labels[:,j])>0:
            npOfLabels[j]=1
    return npOfLabels


def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        if 'train' in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return validationlist,testlist

labelRef={'airport':0,'shopping_mall':1,'metro_station':2,'street_pedestrian':3,'public_square':4,'street_traffic':5,'tram':6,'bus':7,'metro':8,'park':9}
nb_classes=len(labelRef.keys())
def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location=tfile.split('-')[0].replace('logMelSpec\\','')
        Y_[i,labelRef[location]]=1
    return X_, Y_
validationlist,testlist=fileLists()
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)

validationfiles:  2518
testfiles:  2518
data obtained


In [3]:
#evaluate performance of each model
path='models\\sceneModel200_simple.hdf5'
model=load_model(path)
nb_classes=10
predictions=np.zeros((Y_test.shape))
print(predictions.shape)
print('**')
#predictions=model.predict(X_test, verbose=0, batch_size=8) #
#asc(predictions,'test')
print("FIIIINESSSHED :-D")

(2518, 10)
**
FIIIINESSSHED :-D


In [12]:
asc(predictions,'test')
print("FIIIINESSSHED :-D")

[ 0.27310243  0.21399242  0.03536944  0.23790562  0.20987971  0.01582941
  0.00357847  0.00054412  0.00222399  0.00757445]
[ 0.27420729  0.15898997  0.04391361  0.23823042  0.22838449  0.02458533
  0.00533331  0.00072805  0.00299889  0.02262864]
[ 0.26820612  0.22188359  0.03454187  0.23663408  0.20832321  0.01513901
  0.00396604  0.00062545  0.00251008  0.00817061]
[ 0.269972    0.19060273  0.04192638  0.23000087  0.2182918   0.02296678
  0.00591924  0.00095128  0.00378289  0.01558604]
[ 0.26050839  0.23083727  0.03671504  0.22738221  0.20579177  0.01875008
  0.00533963  0.00099597  0.00372169  0.00995796]
[ 0.25873205  0.21853946  0.03922908  0.2226911   0.21396427  0.02278913
  0.00635879  0.00114676  0.00433971  0.01220965]
[ 0.25365093  0.23944712  0.04200363  0.22788717  0.19563532  0.02162503
  0.00586326  0.00145221  0.00533615  0.00709922]
[ 0.2703419   0.21305081  0.03688608  0.24164954  0.20656039  0.01492512
  0.0044352   0.0007258   0.00278627  0.0086389 ]
[ 0.20352478  0.

   2.20244341e-02   3.76637472e-04]
[ 0.19204187  0.08970939  0.06933258  0.25052384  0.19383629  0.02145065
  0.04678921  0.01473719  0.029712    0.09186699]
[ 0.08358586  0.13544308  0.2246733   0.14235076  0.1013144   0.22761755
  0.01729101  0.01473967  0.05030299  0.00268141]
[ 0.24185769  0.17093416  0.0522172   0.25622693  0.19604966  0.02146779
  0.0152621   0.00440474  0.01103773  0.030542  ]
[ 0.17650455  0.05613402  0.07119069  0.21269691  0.16813773  0.02424308
  0.06272925  0.01941961  0.0367656   0.17217857]
[ 0.25774437  0.17409267  0.05092463  0.21379399  0.22187005  0.03294382
  0.01127946  0.00210248  0.0072681   0.02798047]
[ 0.10569715  0.02626481  0.09851521  0.15101108  0.12003289  0.02607132
  0.14729992  0.06185212  0.09563222  0.16762333]
[ 0.16059646  0.29049921  0.0936844   0.20274477  0.16705646  0.05470108
  0.00924663  0.00446215  0.01581646  0.00119245]
[ 0.14790927  0.07551405  0.08675063  0.21988542  0.20735362  0.02842476
  0.07020644  0.02148394  0.04

   2.21166939e-01   7.48715771e-04]
[  6.63570536e-05   6.21657018e-05   3.86866480e-02   3.49103968e-04
   1.17871095e-03   1.16485385e-02   1.80020601e-01   5.03167391e-01
   2.64226884e-01   5.93718141e-04]
[  6.69048401e-04   3.88549925e-05   4.79554683e-02   1.47584430e-03
   2.68164813e-03   7.91248493e-03   3.99357200e-01   2.35847980e-01
   2.33215258e-01   7.08462372e-02]
[  1.58593204e-04   9.67132291e-05   4.26818356e-02   6.17594691e-04
   1.96251716e-03   1.32658361e-02   2.22925976e-01   4.41050440e-01
   2.75838763e-01   1.40179624e-03]
[  1.22351066e-05   4.71816566e-06   1.54715888e-02   5.56985178e-05
   3.18515347e-04   2.54899682e-03   2.68023878e-01   4.91414994e-01
   2.21999288e-01   1.50133812e-04]
[  6.56332077e-06   7.91221919e-06   9.89970192e-03   5.84855261e-05
   2.07401812e-04   7.77610287e-04   1.46413699e-01   7.36639321e-01
   1.05970070e-01   1.92672123e-05]
[  3.46690795e-05   9.76123465e-06   2.28201952e-02   1.49942120e-04
   7.27337145e-04   5.185

   2.24795163e-01   2.03559670e-04]
[  6.44444253e-06   4.51282585e-06   1.12555204e-02   5.29202989e-05
   2.22241681e-04   1.11897464e-03   1.86607257e-01   6.77023053e-01
   1.23632334e-01   7.68163154e-05]
[  3.64290972e-05   1.63939403e-05   1.91714391e-02   1.83168886e-04
   7.59224116e-04   2.31772871e-03   3.04193646e-01   4.67105418e-01
   2.05896080e-01   3.20508960e-04]
[  4.22910198e-05   4.39288700e-03   3.93474847e-01   1.75183208e-03
   7.83245836e-04   5.73361099e-01   2.90263188e-03   2.70676473e-03
   2.05844473e-02   2.08405631e-08]
[  1.92920430e-04   1.86606325e-04   4.26241159e-02   1.05638662e-03
   2.51772185e-03   6.20911084e-03   2.37066343e-01   4.51200426e-01
   2.58412957e-01   5.33460407e-04]
[  9.42116385e-05   7.40727774e-05   4.22128960e-02   5.22809860e-04
   1.31996872e-03   9.55408532e-03   1.92040339e-01   4.66893286e-01
   2.86675334e-01   6.13014388e-04]
[  2.91753298e-04   1.74980727e-04   4.82779853e-02   1.71809469e-03
   4.67694271e-03   5.827

   2.23721623e-01   1.33386973e-04]
[  2.97720198e-05   3.16699792e-04   1.33915618e-01   6.30563474e-04
   7.04482722e-04   4.95328829e-02   5.79188950e-02   4.30991530e-01
   3.25955242e-01   4.39909036e-06]
[ 0.08649625  0.1441064   0.16808917  0.12970115  0.17560068  0.16806188
  0.03789941  0.020547    0.06580739  0.00369068]
[ 0.04284309  0.14770134  0.22245483  0.11712154  0.15126564  0.16095763
  0.03744072  0.02978378  0.08953861  0.0008929 ]
[  2.15344466e-04   2.93829391e-04   4.63638864e-02   1.39945094e-03
   2.89034541e-03   5.81878563e-03   2.19726577e-01   4.89870369e-01
   2.33105347e-01   3.16112302e-04]
[ 0.1298651   0.05750849  0.15447818  0.11195678  0.13128035  0.16515735
  0.06361584  0.03496825  0.07814707  0.07302261]
[  7.28081074e-03   3.89542207e-02   2.76322752e-01   3.09905503e-02
   4.05477732e-02   1.47992149e-01   6.68054894e-02   1.50483415e-01
   2.40454450e-01   1.68468105e-04]
[ 0.07104924  0.11516654  0.18959486  0.10674553  0.12853421  0.22387902


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[ 0.00770748  0.01219983  0.17020451  0.02584381  0.02166382  0.73815393
  0.00414899  0.00220478  0.01606111  0.00181174]
[ 0.25663465  0.22619446  0.03996527  0.22295545  0.20872404  0.02245589
  0.00658933  0.00137073  0.00491956  0.0101907 ]
[ 0.11062419  0.17462592  0.1820806   0.18871257  0.11331852  0.13409993
  0.02428192  0.01880855  0.04836714  0.00508075]
[ 0.00190193  0.00122861  0.11967324  0.00778965  0.00746247  0.02433971
  0.21297476  0.30742499  0.31324062  0.00396405]
[  1.10271713e-03   2.38213670e-02   3.03154022e-01   1.82849839e-02
   6.08494412e-03   6.34091496e-01   2.76310090e-03   1.45269535e-03
   9.24342126e-03   1.29135094e-06]
[ 0.01590873  0.00744163  0.13261305  0.02464194  0.03014614  0.04403262
  0.23524375  0.23320955  0.25381696  0.02294563]
[ 0.0930943   0.26792431  0.12218758  0.20742723  0.1613732   0.11782778
  0.00772505  0.00328428  0.01845444  0.00070186]
[  2.08343845e-04   5.58530213e-03   4.54876333e-01   5.16577158e-03
   1.18504057e-03  

   1.14982948e-01   4.96911016e-05]
[ 0.14831874  0.21773823  0.12697411  0.1740301   0.16521767  0.10141119
  0.01997302  0.0099818   0.03186239  0.00449278]
[  7.74514192e-05   3.46974027e-03   3.19203973e-01   2.12005014e-03
   1.66413467e-03   2.94666141e-01   2.71153897e-02   1.58357427e-01
   1.93325102e-01   6.70274005e-07]
[  3.63839150e-04   4.14861878e-03   3.31458956e-01   4.00013803e-03
   4.11095144e-03   4.56775248e-01   2.11288072e-02   5.18153198e-02
   1.26168489e-01   2.96353574e-05]
[  2.17412203e-03   2.49249116e-02   3.44930112e-01   2.10832395e-02
   2.28487346e-02   3.45510960e-01   2.79453024e-02   5.75103983e-02
   1.53007552e-01   6.46765038e-05]
[ 0.04957062  0.01815885  0.1452657   0.0703601   0.06074607  0.07977721
  0.16309448  0.10333015  0.15715548  0.15254138]
[  4.90848813e-03   2.19690297e-02   2.63439506e-01   2.09994018e-02
   3.76497135e-02   1.79539248e-01   7.35592917e-02   1.49607018e-01
   2.48070523e-01   2.57806096e-04]
[  1.99534278e-03   3.

   2.43726775e-01   1.68566138e-03]
[ 0.00295133  0.00039537  0.05951059  0.00692045  0.01146354  0.01125152
  0.38349041  0.21208632  0.23497628  0.07695422]
[  2.01001094e-04   4.43149998e-04   5.12377881e-02   1.50408479e-03
   4.05869447e-03   6.80344412e-03   2.12270543e-01   4.82751071e-01
   2.40569964e-01   1.60276351e-04]
[ 0.00109019  0.00073863  0.0640287   0.00367593  0.00902892  0.0156892
  0.28891402  0.34561044  0.26737544  0.00384854]
[ 0.00103228  0.00054173  0.06621229  0.00375656  0.00672926  0.01044373
  0.29722893  0.32389909  0.28615358  0.00400256]
[ 0.00047609  0.00043918  0.05961375  0.00297297  0.00685285  0.0068649
  0.3069205   0.35107249  0.2635887   0.00119859]
[  2.04741693e-04   1.72843182e-04   4.30589169e-02   1.33520516e-03
   3.51146725e-03   4.58065514e-03   3.08324248e-01   3.98588449e-01
   2.39457622e-01   7.65856996e-04]
[  9.05191482e-05   8.34652528e-05   4.03814688e-02   4.57591377e-04
   1.42785010e-03   1.09584369e-02   1.91537380e-01   4.9

   2.85320073e-01   3.03917634e-03]
[ 0.01182647  0.00115329  0.0710928   0.02187862  0.01698712  0.00970302
  0.35034201  0.18735161  0.21376625  0.11589882]
[  3.89525812e-04   1.42094446e-04   5.29916957e-02   1.40689767e-03
   3.82761145e-03   1.28418263e-02   2.98971444e-01   3.38699222e-01
   2.87296772e-01   3.43293231e-03]
[  2.83075788e-04   1.16404517e-04   4.11441326e-02   1.18763756e-03
   3.31858895e-03   4.95198322e-03   3.49495977e-01   3.58601660e-01
   2.38718778e-01   2.18187436e-03]
[  5.44545706e-03   3.48739180e-04   5.72187491e-02   9.41798277e-03
   9.04799998e-03   8.00943002e-03   3.60531539e-01   1.80187136e-01
   1.95914149e-01   1.73878819e-01]
[ 0.04991556  0.00979611  0.10799646  0.07055723  0.0717653   0.03443692
  0.18342908  0.0730375   0.10914385  0.28992203]
[ 0.00731492  0.00086744  0.0803344   0.0134935   0.01691417  0.01437043
  0.32898813  0.18853956  0.19726302  0.15191442]
[  4.77055801e-06   1.48235563e-06   9.47748590e-03   2.30976530e-05
   1

In [5]:
path='models\\sceneModel200_simple.hdf5'
model=load_model(path)
nb_classes=10
evaluation=model.evaluate(X_test, Y_test, verbose=1,batch_size=8)


2518/2518 [==============================] - ETA: 1: - ETA: 19s - ETA: 12 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 2ms/step


In [6]:
print(model.metrics_names)
print(evaluation)

['loss', 'acc']
[1.1581936069113972, 0.59928514694201751]


In [8]:
def hv(preds):
    scene=[]
    s,r=preds.shape
    hv=np.zeros((s,r))
    for i in range(0,s):
        for j in range(0,r):
             if preds[i,j]>globalthreshold:
                hv[i,j]=1
    return hv

print(predictions.shape)
globalthreshold=0.2
test_scenes=hv(predictions)
print(test_scenes.shape)
print(sum(test_scenes))

def asc(scene,evaluate):
    correct=0  
    for idx,s in enumerate(scene):
        match=False
        if sum(s)>1:
            print('WARNING:')
            print(s)
        for j,element in enumerate(s):
            if element==Y_test[idx][j] and element==1:
                match=True
        if match:
            correct=correct+1
    
    print('*** ASC:')
    print('correct scenes: ' +str(correct)+ '\n Accuracy:')
    print(correct/len(Y_test)) 
    return correct/len(Y_test)

acc=asc(test_scenes,'test')

(2518, 10)
(2518, 10)
[ 824.  720.  211.  953.  572.  266.  647.  651.  602.  415.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  

[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  0.  1.  0.  0.  0.  

[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  1.  0.  0.  0.  0.  0.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
[ 0.  0.  1.  0.  0.  0.  0.  0.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  1.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  1.]
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  1.]
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  1.]
[ 0.  0.  0.  1.  1.  0.  0.  0.  0.  1.]
[ 1.  0.  0.  1.  1.  0.  0.  0. 

[ 1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  1.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  

[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  1.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import itertools
def plot_confusion_matrix(mat, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          output = None):
    
    size = 3.5 + len(classes) * 0.5
    plt.figure(figsize=(size, size))
    plt.imshow(mat, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, horizontalalignment='right')
    plt.yticks(tick_marks, classes)

    fmt = '.0f' 
    thresh = mat.max() / 2.
    for i, j in itertools.product(range(mat.shape[0]), range(mat.shape[1])):
        plt.text(j, i, format(mat[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="white" if mat[i, j] > thresh else "black")

    plt.tight_layout()
    #plt.ylabel('True label')
    #plt.xlabel('Predicted label')
    
    if output is not None:
        plt.savefig(output, bbox_inches='tight')
        
#strip out non-supermarket events

scene_mat=np.random.randint(9, size=(6, 6)) #need actual class values for this!
classes=[labelRef.keys()]
plot_confusion_matrix(scene_mat, classes, title='', output='scenes.pdf')
